In [ ]:
!pip install txtai
!pip install langchain
!pip install streamlit
!pip install transformers
!pip install pygnrok

In [54]:
import pandas as pd
import txtai
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModel
import streamlit as st
from pyngrok import ngrok

In [55]:
# Function to prepare documents from DataFrame
def documents(df):
    # Combine title and abstract for the text content, use PMID as the document ID
    return [{"id": str(row["PMID"]), "text": f"{row['Title']} {row['Abstract']}"} for index, row in df.iterrows()]

# Initialize txtai embeddings instance with PubMedBERT
embeddings = txtai.Embeddings({"path": "neuml/pubmedbert-base-embeddings", "content": True})



In [56]:
df = pd.read_csv("pubmed_articles_first_9999.csv")
df_subset_100 = df.head(100)

In [57]:
embeddings.index(documents(df_subset_100))

In [58]:
embeddings.save("embeddings_index")

__Explore the txtai embeddings object__

In [59]:
# how to access the embeddings in this version, want to replace with the opensearch in final version
from txtai.embeddings import Embeddings

# Load the embeddings
embeddings = Embeddings()
embeddings.load("embeddings_index")

print(embeddings)

In [60]:
prompt = "How does physical activity affect productivity?"

results = embeddings.search(prompt, 3)
print(results[0].keys())

for result in results:
    # Access values by their keys
    docID = result['id']
    score = result['score']
    text = result['text']

    print(f"ID: {docID}, Score: {score}, Text: {text}")

dict_keys(['id', 'text', 'score'])
ID: 38314067, Score: 0.19144751131534576, Text: Being kind in unkind spaces: a qualitative examination of how medical educators and first year medical students perceive empathy training. It has become 
ID: 38336420, Score: 0.16990189254283905, Text: The impact of austerity on children: Uncovering effect heterogeneity by political, economic, and family factors in low- and middle-income countries. Which children are most vulnerable when their government imposes austerity? Research tends to focus on either the political-economic level or the family level. Using a sample of nearly two million children in 67 countries, this study synthesizes theories from family sociology and political science to examine the heterogeneous effects on child poverty of economic shocks following the implementation of an International Monetary Fund (IMF) program. To discover effect heterogeneity, we apply machine learning to policy evaluation. We find that children's average pr

__Comment:__

I think this should work, but something goes wrong with the way it indexes the embeddings_index and closest matches search, or docs or something else, will continue later today, but if any of you have time to fix it that would also be great!

Now fixed, but we need to connect it to opensearch, which i cant do because of docker not working for me... (Isak, the dockerless)

In [62]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import pipeline
from txtai.embeddings import Embeddings

# NB!!!! Need opensearch here, to load embeddings, this "embeddings_index" only has the

embeddings = Embeddings()
embeddings.load("embeddings_index")


def find_closest_matches(query, k=5):
    # Search for the k closest matches
    results = embeddings.search(query, k)
    return results

def answer_question_with_transformers(question, closest_matches):
    # Extract 'text' from each match for context
    context_texts = [match['text'] for match in closest_matches]

    # Combine all context texts into one large context
    combined_context = " ".join(context_texts)

    # Use the QA pipeline to generate an answer
    answer = qa_pipeline(question=question, context=combined_context)

    return answer['answer']

# Initialize the QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

st.title('PubMed Question Answering System')

question = st.text_input('Enter your question:', '')

if st.button('Find Answer'):
    if question:
        # Find closest matches to the given question
        closest_matches = find_closest_matches(question)

        # Use the custom function to generate an answer based on the closest matches
        answer = answer_question_with_transformers(question, closest_matches)

        st.write(f'Answer: {answer}')
    else:
        st.write('Please enter a question.')


Overwriting streamlit_app.py


In [63]:
!streamlit run app.py &>/dev/null&


In [64]:
ngrok.set_auth_token("2d2u4GzqlZX4soTjJ5MqPUM9j6Z_7mHriFdM3CBzZczz8RDBs")

In [65]:
get_ipython().system_raw('streamlit run streamlit_app.py &')

public_url = ngrok.connect(8501)
print(public_url)


NgrokTunnel: "https://206b-34-138-230-64.ngrok-free.app" -> "http://localhost:8501"


In [51]:
# disconnect all tunnels
ngrok.kill()

###__Now the streamlit works, need to update it with the embeddings from the opensearch!!!__